In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existent cache directory
fc-cache: succeeded


In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', my_path)
sys.path.insert(0,my_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileExistsError: ignored

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic') 

In [ ]:
# 사용할 라이브러리 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
%matplotlib inline
import warnings

warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/LMEMBERS 프로젝트(1차)/dataset/기준_데이터셋1.csv',encoding='cp949',index_col=0)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19147 entries, 0 to 19146
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   고객번호     19147 non-null  int64  
 1   성별       19147 non-null  object 
 2   연령대      19147 non-null  object 
 3   14년 상반기  19147 non-null  int64  
 4   14년 하반기  19147 non-null  int64  
 5   15년 상반기  19147 non-null  int64  
 6   15년 하반기  19147 non-null  int64  
 7   1P       19147 non-null  float64
 8   2P       19147 non-null  float64
 9   3P       19147 non-null  float64
dtypes: float64(3), int64(5), object(2)
memory usage: 1.6+ MB


In [ ]:
data_copy = data.copy()

In [ ]:
data['1P'] = data['14년 하반기']/data['14년 상반기'] - 1
data['2P'] = data['15년 상반기']/data['14년 하반기'] - 1
data['3P'] = data['15년 하반기']/data['15년 상반기'] - 1

In [ ]:
data.head()

,고객번호,성별,연령대,14년 상반기,14년 하반기,15년 상반기,15년 하반기,1P,2P,3P
0,1,M,60대,19925355,34116312,17898365,13102518,0.712206,-0.475372,-0.267949
1,2,M,60대,24323230,21922100,27791610,25843820,-0.098718,0.267744,-0.070086
2,3,M,60대,851215,1042554,592280,679118,0.224783,-0.431895,0.146616
3,4,F,60대,5042106,4257510,3656937,3737257,-0.155609,-0.141062,0.021964
4,5,M,60대,5460150,2225730,2934930,2616260,-0.592368,0.318637,-0.108578


In [ ]:
data3P = data[['고객번호','1P','2P','3P']]
data3P.to_csv('/content/drive/MyDrive/LMEMBERS 프로젝트(1차)/dataset/고객_3P.csv', encoding='cp949')

In [ ]:
data_amount = pd.read_csv('/content/drive/MyDrive/LMEMBERS 프로젝트(1차)/dataset/ppcd_df.csv',encoding='cp949', index_col=0)

In [ ]:
data_amount['고객번호'].nunique()

19383

In [ ]:
# 기수별 제휴사별 구매금액

data_amount.drop(['영수증번호','소분류코드','구매시간','소분류명','대분류명','뉴중분류명','성별','연령대','거주지역'],axis=1,inplace=True)

In [ ]:
data_amount_15 = data_amount[data_amount['구매일자'] > 20150000]
data_amount_14 = data_amount[data_amount['구매일자'] < 20150000]

data_amount_14_uq = data_amount_14[data_amount_14['구매일자'] < 20140700]
data_amount_14_dq = data_amount_14[data_amount_14['구매일자'] > 20140700]
data_amount_15_uq = data_amount_15[data_amount_15['구매일자'] < 20150700]
data_amount_15_dq = data_amount_15[data_amount_15['구매일자'] > 20150700]

In [ ]:
data_amount_14_uq.drop('구매일자',axis=1,inplace=True)
data_amount_14_dq.drop('구매일자',axis=1,inplace=True)
data_amount_15_uq.drop('구매일자',axis=1,inplace=True)
data_amount_15_dq.drop('구매일자',axis=1,inplace=True)

In [ ]:
uq14_ent_amount = data_amount_14_uq.groupby(['고객번호','제휴사']).sum().reset_index()
dq14_ent_amount = data_amount_14_dq.groupby(['고객번호','제휴사']).sum().reset_index()
uq15_ent_amount = data_amount_15_uq.groupby(['고객번호','제휴사']).sum().reset_index()
dq15_ent_amount = data_amount_15_dq.groupby(['고객번호','제휴사']).sum().reset_index()

In [ ]:
uq14_ent_amount.head()

,고객번호,제휴사,구매금액
0,1,A,19703210
1,1,B,21800
2,1,C,200345
3,2,A,24292730
4,2,B,30500


In [ ]:
a = pd.pivot_table(data=uq14_ent_amount,values='구매금액',index='고객번호',columns='제휴사')
b = pd.pivot_table(data=dq14_ent_amount,values='구매금액',index='고객번호',columns='제휴사')
c = pd.pivot_table(data=uq15_ent_amount,values='구매금액',index='고객번호',columns='제휴사')
d = pd.pivot_table(data=dq15_ent_amount,values='구매금액',index='고객번호',columns='제휴사')

In [ ]:
a.rename(columns={'A':'구매금액_1_A','B':'구매금액_1_B','C':'구매금액_1_C','D':'구매금액_1_D'},inplace='True')
b.rename(columns={'A':'구매금액_2_A','B':'구매금액_2_B','C':'구매금액_2_C','D':'구매금액_2_D'},inplace='True')
c.rename(columns={'A':'구매금액_3_A','B':'구매금액_3_B','C':'구매금액_3_C','D':'구매금액_3_D'},inplace='True')
d.rename(columns={'A':'구매금액_4_A','B':'구매금액_4_B','C':'구매금액_4_C','D':'구매금액_4_D'},inplace='True')

In [ ]:
a = a.reset_index()
b = b.reset_index()
c = c.reset_index()
d = d.reset_index()

In [ ]:
data_copy

,고객번호,성별,연령대,14년 상반기,14년 하반기,15년 상반기,15년 하반기,1P,2P,3P
0,1,M,60대,19925355,34116312,17898365,13102518,0.712206,-0.475372,-0.267949
1,2,M,60대,24323230,21922100,27791610,25843820,-0.098718,0.267744,-0.070086
2,3,M,60대,851215,1042554,592280,679118,0.224783,-0.431895,0.146616
3,4,F,60대,5042106,4257510,3656937,3737257,-0.155609,-0.141062,0.021964
4,5,M,60대,5460150,2225730,2934930,2616260,-0.592368,0.318637,-0.108578
...,...,...,...,...,...,...,...,...,...,...
19142,19356,F,40대,170160,1536090,2104968,2434587,8.027327,0.370342,0.156591
19143,19357,F,40대,91950,3281346,3044944,2246675,34.686199,-0.072044,-0.262162
19144,19359,M,50대,43190,1286327,837240,1092380,28.782982,-0.349124,0.304739
19145,19360,F,40대,368050,2060040,1412140,886060,4.597174,-0.314508,-0.372541


In [ ]:
data_inter4 = pd.merge(data_inter3,d,how='inner',on='고객번호')

In [ ]:
data_inter5 = data_inter4.drop('index',axis=1)

In [ ]:
data_inter5.to_csv('/content/drive/MyDrive/LMEMBERS 프로젝트(1차)/dataset/기준_3P_분기제휴사_구매금액_1006.csv', encoding='cp949')

In [ ]:
data_inter5

,고객번호,성별,연령대,14년 상반기,14년 하반기,15년 상반기,15년 하반기,1P,2P,3P,구매금액_1_A,구매금액_1_B,구매금액_1_C,구매금액_1_D,구매금액_2_A,구매금액_2_B,구매금액_2_C,구매금액_2_D,구매금액_3_A,구매금액_3_B,구매금액_3_C,구매금액_3_D,구매금액_4_A,구매금액_4_B,구매금액_4_C,구매금액_4_D
0,1,M,60대,19925355,34116312,17898365,13102518,0.712206,-0.475372,-0.267949,19703210.0,21800.0,200345.0,NaN,33340300.0,376140.0,399872.0,NaN,17382750.0,208136.0,307479.0,NaN,12800890.0,184900.0,116728.0,NaN
1,2,M,60대,24323230,21922100,27791610,25843820,-0.098718,0.267744,-0.070086,24292730.0,30500.0,NaN,NaN,21875650.0,46450.0,NaN,NaN,27486650.0,132500.0,NaN,172460.0,25799820.0,NaN,NaN,44000.0
2,3,M,60대,851215,1042554,592280,679118,0.224783,-0.431895,0.146616,80000.0,NaN,771215.0,NaN,NaN,6960.0,1035594.0,NaN,NaN,NaN,592280.0,NaN,NaN,NaN,679118.0,NaN
3,4,F,60대,5042106,4257510,3656937,3737257,-0.155609,-0.141062,0.021964,4628260.0,80060.0,333786.0,NaN,3711520.0,420740.0,125250.0,NaN,3170780.0,333864.0,152293.0,NaN,2764050.0,759402.0,213805.0,NaN
4,5,M,60대,5460150,2225730,2934930,2616260,-0.592368,0.318637,-0.108578,5444150.0,16000.0,NaN,NaN,2225730.0,NaN,NaN,NaN,2917380.0,NaN,NaN,17550.0,2586960.0,NaN,NaN,29300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19142,19356,F,40대,170160,1536090,2104968,2434587,8.027327,0.370342,0.156591,NaN,NaN,170160.0,NaN,29000.0,17200.0,1489890.0,NaN,NaN,NaN,2104968.0,NaN,211900.0,NaN,2222687.0,NaN
19143,19357,F,40대,91950,3281346,3044944,2246675,34.686199,-0.072044,-0.262162,52700.0,39250.0,NaN,NaN,1052250.0,2223486.0,5610.0,NaN,823900.0,2206144.0,14900.0,NaN,344000.0,1902675.0,NaN,NaN
19144,19359,M,50대,43190,1286327,837240,1092380,28.782982,-0.349124,0.304739,NaN,NaN,43190.0,NaN,NaN,NaN,1286327.0,NaN,NaN,4450.0,832790.0,NaN,NaN,NaN,1092380.0,NaN
19145,19360,F,40대,368050,2060040,1412140,886060,4.597174,-0.314508,-0.372541,NaN,NaN,NaN,368050.0,10300.0,58610.0,NaN,1991130.0,290100.0,NaN,NaN,1122040.0,334600.0,NaN,NaN,551460.0
